# League of Conservation Voters - Scorecard Download

Data: https://www.lcv.org/congressional-scorecard/members-of-congress/

In [ ]:
import os
import io
import requests
import zipfile
import tempfile
import shutil
import geopandas as gpd
import ibis
from ibis import _
from cng.utils import *
from cng.h3 import *

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
con.raw_sql("SET THREADS=100;")
set_secrets(con)

bucket = "public-election"
s3_prefix = "league_of_conservation_voters/members_of_congress"

In [ ]:
import geopandas as gpd
def download_csv(bucket,s3_prefix,chamber):
    for year in range(1988,2025):
        # print(year)
        url=f'https://www.lcv.org/congressional-scorecard/members-of-congress/?export=true&sort=state-a-z&active_tab=&chamber={chamber}&street_address=&zip=&session_year={year}&state=&party=All&last_name=&export-type=moc-listing'
        csv_file=f'{chamber}/lcv_scorecard_{year}.csv'
        save_url=f's3://{bucket}/{s3_prefix}/{csv_file}'
        
        # format is really weird, adding a new column with the state 
        t= (con.read_csv(url,skip=1, delim=',', quote='"',header=True,null_padding=True)
            .mutate(rid=ibis.row_number(),
                    state_raw =ibis.case()
                    .when(_["Last Name"].isnull() & _.Party.isnull(), _["First Name"])
                    .else_(None)
                    .end()
                   )
            .mutate(
                state_group=ibis.ifelse(_.state_raw.notnull(), _.rid, None)
                    .max()
                    .over(ibis.window(order_by=_.rid, preceding=None, following=0))
            )
            .mutate(state=_.state_raw.max().over(ibis.window(group_by="state_group")))
            .filter(_["Last Name"].notnull())
            .mutate(year=year, chamber=ibis.literal(chamber))
            .rename(first_name="First Name",last_name="Last Name",party="Party",district="District",year_score="Year Score",lifetime_score="Lifetime Score")
            .filter(_.year_score!="Year Score")
            .mutate(year_score=_.year_score.substitute({'na':ibis.null()}))
            .mutate(lifetime_score=_.lifetime_score.substitute({'na':ibis.null()}))
            .cast({"year_score": "int", "lifetime_score": "int"})
            .select('year','chamber','state','first_name','last_name','party','district','year_score','lifetime_score')
            .order_by('state')
        )
        t.to_csv(save_url)



In [ ]:
download_csv(bucket,s3_prefix,chamber='senate')

In [ ]:
download_csv(bucket,s3_prefix,chamber='house')